# Python wrapper for bcftools
Contains additional functions to mimic output of HAPSBURG for downstream analysis
@Harald Ringbauer, October 2019

In [1]:
import pandas as pd
import numpy as np
import socket
import os as os
import sys as sys
import multiprocessing as mp
import h5py

socket_name = socket.gethostname()
print(socket_name)
if socket_name == "VioletQueen":
    path = "/home/harald/git/HAPSBURG/"   # The Path on Harald's machine
elif socket_name.startswith("midway2"):
    print("Midway jnovmbre partition detected.")
    path = "/project2/jnovembre/hringbauer/HAPSBURG/"  # The Path on Midway Cluster
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")
    
os.chdir(path)  # Set the right Path (in line with Atom default)

sys.path.append("./PackagesSupport/h5_python/")
from h5_functions import hdf5_to_vcf, load_h5   # Import Function to convert hdf5 to vcf
sys.path.append("./PackagesSupport/parallel_runs/")
from helper_functions import prepare_path, create_folders, postprocess_iid  # To split up ground truth ROH

print(os.getcwd()) # Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(f"CPU Count: {mp.cpu_count()}")

VioletQueen
/home/harald/git/HAPSBURG
CPU Count: 4


### Functions to run VCF Tools for a single sample

In [2]:
def post_process_bcftools(outfile):
    """Post Process bcftools output"""
    outtxt = outfile + ".txt"
    outST = outfile + "ST.txt"
    outRG = outfile + "RG.txt"
    
    !grep ^ST $outtxt > $outST
    !grep ^RG $outtxt > $outRG
    #!rm $outtxt # Clean up the now redundant original output
    
    df_pos = pd.read_csv(outST, sep='\t', header=None, usecols=range(1,6))
    df_pos.columns = ["iid", "ch", "pos", "state", "qual"]
    df_rohs = pd.read_csv(outRG, sep='\t', header=None, usecols=range(1,8))
    df_rohs.columns = ["iid", "ch", "Start", "End", "length", "markers", "qual"]
    return df_pos, df_rohs

def run_bcftools_roh(vcf_file, outfile, mp="./Data/1000Genomes/Markers/rec_map_bcf.chr3.txt",
                    af="./Data/1000Genomes/Markers/af_1000G_EUR_bcf.chr3.txt.gz", pl=False):
    """Run PLINK ROH Caller on path_vcf, and save results in outfile.txt.
    Uses Map File mp and Allele Frequency File AF (prepared in prep_map_af_bcftools.ipynb)
    Return 2 Dataframes (per site,  total roh blocks )
    pl: Whether to use Genotype Likelihoods"""
    outtxt = outfile + ".txt"
    if pl==False:
        !bcftools roh -G30 --AF-file $af -m $mp $vcf_file > $outtxt
        
    elif pl==True:
        !bcftools roh --AF-file $af -m $mp $vcf_file > $outtxt
        
    # -V 1e-10   ### Command to do Viterbi Training
    
def create_hapsburg_df(df_t, map_dct):
    """Modify bcftools output to HAPSBURG format.
    Return right Dataframe"""
    df_t["StartM"] = df_t["Start"].map(map_dct)
    df_t["EndM"] = df_t["End"].map(map_dct)
    df_t["lengthM"] = df_t["EndM"] - df_t["StartM"]

    # Add all fields for roh.csv
    df_t = df_t[["Start", "End", "StartM", "EndM", "length", "lengthM", "iid", "ch"]]
    return df_t

### Quick Single Example Run on one VCF

In [14]:
%%time
vcf_file = "./Simulated/1000G_Mosaic/TSI5/ch3_6cm/data.vcf"
outfile  = "./Diverse/nosync/bcfroh_out"

run_bcftools_roh(vcf_file, outfile, pl=False)
#df_pos, df_rohs = post_process_bcftools(outfile)

#df_rohs = create_hapsburg_df(df_rohs, map_dct)

Number of target samples: 100
Number of --estimate-AF samples: 0
Number of sites in the buffer/overlap: unlimited
Number of lines overlapping with --AF-file/processed: 77652/0
Number of lines filtered/no AF/not biallelic/dup: 0/77651/0/1
No usable sites were found.
CPU times: user 14.9 ms, sys: 24.6 ms, total: 39.5 ms
Wall time: 789 ms


In [3]:
#############################################
### Combine all subfunctions

def full_bcftools_mosaic(input_base_folder, ch=3, prefix_out="bcftools/", convert_h5=True):
    """Run PLINK on Mosaic Data Set in ./Simulated"""
    
    print("Converting HDF5 into VCF...")
    input_h5, input_vcf, bcf_folder = create_folders(input_base_folder, outfolder="bcf_out/")
    if convert_h5:
        hdf5_to_vcf(input_h5, input_vcf, chrom=ch) # Convert to VCF
    
    print("Running bcftools/ROH...")
    outfile = bcf_folder + "bcfroh_out"
    run_bcftools_roh(input_vcf, outfile)   # Run BCF tools on VCF
    df_pos, df_rohs = post_process_bcftools(outfile) # Load the output Data
    
    ### Create the Mapping Dictionary
    print("Creating Map Dict...")
    f = load_h5(path=input_h5, output=False)
    map_dct = dict(zip(f["variants/POS"], f["variants/MAP"]))
    iids = f["samples"][:] # Get the IIDs
    
    print("Splitting up BCF results and GT...")
    df_rohs = create_hapsburg_df(df_rohs, map_dct)
    postprocess_iid(df_rohs, input_base_folder, iids, ch, prefix_out)
    print(f"Finished {len(iids)} Individuals!")

### Run bcftools on all Individuals for Mosaic Folder

In [31]:
%%time
full_bcftools_mosaic(input_base_folder = "./Simulated/1000G_Mosaic/TSI5/ch3_6cm",
                     ch=3, prefix_out="bcftools/", convert_h5=False)

Converting HDF5 into VCF...
Running bcftools/ROH...
Number of target samples: 100
Number of --estimate-AF samples: 0
Number of sites in the buffer/overlap: unlimited
Number of lines total/processed: 77652/70453
Creating Map Dict...
Splitting up BCF results and GT...
Finished 100 Individuals!
CPU times: user 18.2 s, sys: 750 ms, total: 19 s
Wall time: 28.5 s


### Run multiple lengths of copied in Chromosomes

In [ ]:
base_path =  "./Simulated/1000G_Mosaic/TSI5/"

for l in [0, 2, 4, 6, 8, 10]:
    input_base_folder = base_path + f"ch3_{l}cm"
    print(f"\nDoing ROH bcftools on {input_base_folder}...")
    full_bcftools_mosaic(input_base_folder, ch=3, prefix_out="bcftools/")

## Split up the Posterior Output for Mosaic Folders
Run to split map.csv and posterior0.csv into bcftools/ output folder

In [23]:
def post_process_postbcf(basepath, map_dct):
    """Post Process the Posterior of bcftools"""
    df_t = pd.read_csv(basepath + "bcf_out/bcfroh_outST.txt", header=None, sep="\t")
    df_t.drop(columns=0, inplace=True) # Get rid of the first component
    df_t.columns = ["iid", "chr", "pos", "state", "post"]

    ### Transfrom it from PHRED scale to posterior
    p = 10**(-df_t["post"]/10)  # prob for alternative state
    post = p * df_t["state"] + (1-p) * (1 - df_t["state"])   # 1 ROH State 0 HW
    df_t["post"] = np.log(post.values + 1e-10)
    df_t["map"] = df_t["pos"].map(map_dct)
    return df_t

def split_up_bcftools_post(basepath, df_bcf, iid, ch, prefix_out="bcftools/"):
    """Split up postprocessed Dataframe of bcftools/ROH output into
    Mosaic folders. Save map and posterior file"""
    output_base_folder = os.path.join(basepath, "output/")
    pathout = prepare_path(output_base_folder, iid, ch=3, prefix_out=prefix_out, logfile=False)
    df_t = df_bcf[df_bcf["iid"] == iid]
    
    df_map = df_t["map"]
    mappath = os.path.join(pathout,"map.csv")
    df_map.to_csv(mappath, sep=",", index=None, header=None)
    
    df_pos = df_t["post"]
    postpath = os.path.join(pathout,"posterior0.csv")
    df_pos.to_csv(postpath, sep=",", index=None, header=None)
    print(f"Saved Posterior to {postpath}")

In [ ]:
### Merge in Map(Takes about 10s)
### Create the Mapping Dictionary (Run in Wrapper Function)
input_h5 =  "./Simulated/1000G_Mosaic/TSI5/ch3_6cm/data.h5"

print("Creating Map Dict...")
f = load_h5(path = input_h5, output=False)
map_dct = dict(zip(f["variants/POS"], f["variants/MAP"]))

# Run with Genotype Likelihoods (PL field)

### Quick test on single file

In [10]:
%%time
path_vcf = "./Simulated/1000G_Mosaic/TSI5/lambda_rc1.0/ch3_4cm/data.vcf"
outfile  = "./Diverse/bcfroh_out"

run_bcftools_roh(path_vcf, outfile, pl=False)
#df_pos, df_rohs = post_process_bcftools(outfile)

Number of target samples: 100
Number of --estimate-AF samples: 0
Number of sites in the buffer/overlap: unlimited
Number of lines overlapping with --AF-file/processed: 77650/0
Number of lines filtered/no AF/not biallelic/dup: 0/77649/0/1
No usable sites were found.
CPU times: user 17.8 ms, sys: 32.2 ms, total: 50 ms
Wall time: 1.42 s


# Area 51
Area to test code

### Split up the posterior from bcftools

### Convert to VCF, storing the Genotype Likelihood

In [4]:
%%time
### Test for transformation to vcf with PL
path_h5 = "./Simulated/1000G_Mosaic/TSI5/lambda_rc1.0/ch3_4cm/data.h5"
path_vcf = "./Simulated/1000G_Mosaic/TSI5/lambda_rc1.0/ch3_4cm/data.vcf"

hdf5_to_vcf(path_h5, path_vcf, iids=[], markers=[], chrom=3, pl_field=True)

Loaded HDF5
Loaded 77650 variants
Loaded 100 individuals
['AD', 'GT']
['ALT', 'MAP', 'POS', 'REF']
Calculating Genotype Likelihoods...
Saving to VCF...
Successfully saved VCF to ./Simulated/1000G_Mosaic/TSI5/lambda_rc1.0/ch3_4cm/data.vcf
CPU times: user 1min 5s, sys: 4.83 s, total: 1min 10s
Wall time: 1min 10s


In [20]:
 f= load_h5(path_h5)

Loaded HDF5
Loaded 77650 variants
Loaded 100 individuals
['AD', 'GT']
['ALT', 'MAP', 'POS', 'REF']


In [21]:
f["calldata/GT"][0,:10,:]

array([[0, 0],
       [1, 0],
       [1, 1],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 0]], dtype=int32)

In [22]:
f["calldata/AD"][0,:10,:]

array([[1, 0],
       [0, 0],
       [0, 2],
       [1, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0]], dtype=int32)